# Reglas de Acceso desde dentro y fuera de una cuenta. Que es Root?

Especificamente, que significa lo siguiente, cuando se utiliza como Principal en una politica de recurso de IAM?

"Principal": {"AWS": ["arn:aws:iam::111122223333:root]}

## Introduccion

Este lab examina la diferencia entre las politicas de IAM y las basadas en recursos de AWS. Especificamente, queremos entender
la logica de evaluacion de politicas para los baldes de S3 que permiten el acceso entre cuentas. Para un recordatorio de lo basico de IAM, ver
[Logica de Evaluacion de Politicas de Referencia](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_evaluation-logic.html),
lo cual es valido para cuando el Principal de IAM y el recurso de S3 estan en la misma cuenta de AWS.
 
Para resumir, si una accion esta permitida por una politica basada en identidad, una politica basada en un recurso, o ambas,
entonces AWS permite la accion. Un "Deny" explicito en cualquiera de estas politicas sobreescribe ese permiso.

La situacion cambia para el [accesso entre cuentas](https://aws.amazon.com/premiumsupport/knowledge-center/cross-account-access-s3/).
En este caso, el acceso tiene que ser explicitamente permitido tanto en la politica de acceso del Principal como en la politica del recurso.
Desafortunadamente, el link no menciona el "problema del diputado confundido" para el acceso entre cuentas, el cual ocurre cuando la cuenta
confiada es un vendedor tercero de SaaS (Software como un Servicio, en ingles). Como resultado de esto, muchos vendedores que operan en
baldes de S3 de sus clientes, lo hacen inseguramente


Para este lab, asumiremos que ambas cuentas de AWS son de la misma entidad y dejaremos los problemas de diputado
confundido para el Lab 4 - Acceso Directo vs Asumir Rol: Otorgar acceso entre cuentas a recursos.

<img src="s3-cross-account.png" class="left"/>

Otorgar permisos al Principal-A para acceder al Recurso-B, cuando ambos estan en la misma cuenta, puede hacerse asignando una politica de permisos a dicho Principal. Alternativamente, el acceso entre cuentas se podria otorgar en una politica de recurso tal como la siguiente politica de balde.

Los recursos para este lab son dos baldes de S3, a los que nos referiremos como mybucket1 y mybucket2 (en realidad son mybucket-$random).
Todos los roles mostrados a continuacion tendran la misma politica de permisos que les permitira acceder a recursos genericos de S3

Este lab requiere de un Admin_A y un Admin_B, quienes pueden crear los recursos necesarios. Para nuestros propositos de instruccion, crearemos los recursos a medida que los necesitamos. Si se esta utilizando esto para estudiantes, el administrador puede correr setupA.sh y setupB.sh.

In [ ]:
import json

In [ ]:
%%bash --out bash_output
echo $RANDOM

In [ ]:
rand = str(bash_output.strip())
rand

In [ ]:
rand = str(123)

Para este lab, se necesitan credenciales de administrador en dos cuentas de AWS (A y B), con sus respectivos bloques en ~/.aws/credentials. Algo como:
```
[perfilA]
aws_access_key_id=AKIA**********
aws_secret_access_key=xxxxxxxxxxxxxxxx

[perfilB]
aws_access_key_id=AKIA**********
aws_secret_access_key=xxxxxxxxxxxxxxxx
```

Rellena los valores que quieres usar para perfilA y perfilB debajo. Deben ser los mismos que en tu archivo ~/.aws/credentials.

In [ ]:
perfilA = "chinoAdmin"
perfilB = "chino"
mybucket1 = "mybucket1-" + rand
mybucket2 = "mybucket2-" + rand
path = "/aws-labs/" 
#path actua como un prefijo para los roles y politicas de IAM. 
#Puede ser usado como etiqueta para listar nuestros bienes.

In [ ]:
!aws --profile $perfilA sts get-caller-identity

In [ ]:
!aws --profile $perfilB sts get-caller-identity

Usar los ARN de las dos llamadas anteriores para rellenar debajo

In [ ]:
principalA_arn = "Aqui"
principalB_arn = "Y aqui"
accountA = principalA_arn.split(':')[4]
accountB = principalB_arn.split(':')[4]
accountB

Eso funciona. Sin embargo...

In [ ]:
%%bash
aws --profile "${perfilA}" sts get-caller-identity
aws --profile "${perfilB}" sts get-caller-identity
# Lo siguiente evalua $perfilA (o B) como null
# aws --profile $perfilA sts get-caller-identity
# aws --profile $perfilB sts get-caller-identity

Como puedes ver arriba, debemos ir con cuidado, ya que la magia de celda "bash" puede no funcionar como esperamos. La salida de la celda anterior deberia corresponder a los dos perfiles, pero no lo hace. La moraleja es, no intentar cosas muy elaboradas con bash en jupyter.

## Setup para Administrador

Crear los 4 roles para el lab

In [ ]:
rolA1 = "rolA1-" + rand
rolA2 = "rolA2-" + rand
rolB1 = "rolB1-" + rand
rolB2 = "rolB2-" + rand

Crear ambas politicas "assume", utilizadas para asumir el rol

In [ ]:
# assume_A_policy.json
assume_A_policy = {
  "Version": "2012-10-17",
  "Statement": 
    {
      "Sid": "AssumeRolePolicyForS3ReaderRoleByExternal",
      "Effect": "Allow",
      "Principal": {"AWS": principalA_arn},
      "Action": "sts:AssumeRole"
    }
}

In [ ]:
# assume_B_policy.json
assume_B_policy = {
    "Version": "2012-10-17",
    "Statement": {
        "Sid": "AssumeRolePolicyB",
        "Effect": "Allow",
        "Principal": { "AWS": principalB_arn },
        "Action": "sts:AssumeRole"
    }
}

In [ ]:
assume_A_policy_str = "'" + json.dumps(assume_A_policy) + "'"
assume_B_policy_str = "'" + json.dumps(assume_B_policy) + "'"

Crear los roles dentro de las cuentas

In [ ]:
!aws --profile $perfilA iam create-role --role-name $rolA1 \
                                        --path $path \
                                        --assume-role-policy-document $assume_A_policy_str

In [ ]:
!aws --profile $perfilA iam create-role --role-name $rolA2 \
                                        --path $path \
                                        --assume-role-policy-document $assume_A_policy_str

In [ ]:
!aws --profile $perfilB iam create-role --role-name $rolB1 \
                                        --path $path \
                                        --assume-role-policy-document $assume_A_policy_str

In [ ]:
!aws --profile $perfilB iam create-role --role-name $rolB2 \
                                        --path $path \
                                        --assume-role-policy-document $assume_A_policy_str

Crear los baldes para el lab. Notar que, a diferencia de los recursos creados de IAM, el ARN del balde no incluye el ID de cuenta.

In [ ]:
!aws --profile $perfilA s3api create-bucket --bucket $mybucket1
!aws --profile $perfilA s3api create-bucket --bucket $mybucket2 \
                --create-bucket-configuration LocationConstraint=us-west-1

Copiar algunos archivos de demonstracion dentro de cada balde

In [ ]:
!aws --profile $perfilA s3 cp demo-vars.sh s3://$mybucket1/
!aws --profile $perfilA s3 cp assume_A_policy.json s3://$mybucket2/

Chequear que los archivos hayan sido copiados exitosamente, solo por si acaso

In [ ]:
!aws --profile $perfilA s3 ls s3://$mybucket1
!aws --profile $perfilA s3 ls s3://$mybucket2

Crear un tagset, y añadirlo a los baldes

In [ ]:
tagset = 'TagSet=[{Key=path,Value=' + path + '},{Key=rand,Value=' + rand + '}]'
tagset

In [ ]:
!aws --profile $profileA s3api put-bucket-tagging --bucket $mybucket1 \
                --tagging $tagset
!aws --profile $profileA s3api put-bucket-tagging --bucket $mybucket2 \
                --tagging $tagset

Hemos escrito una funcion llamada awsas, que esta dentro del script aws_run_as.sh en la repo. Puede ser llamado de la siguiente manera:

awsas --profile perfil rol [comandos de aws]

Esto utiliza las credenciales en "perfil" para assumir "rol" y luego correr comandos. Sin esta funcion, deberia utilizarse "sts assume-role", y luego utilizar la salida de eso para crear un perfil nuevo en nuestro archivo ~/.aws/credentials cada vez que quisieramos hacer algo, como descrito [aqui](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-role.html). Este script es muy reciente y esta aun en beta, por lo que si algo no funciona, intentarlo en una shell de bash.

Probemoslo!

In [ ]:
!awsas --profile $perfilA $rolA1 sts get-caller-identity

Esto deberia retornar un rol asumido con el nombre rolA1-$rand, la cuenta en la que se encuentra, y su ARN. Si no lo hace, checkear credenciales y privilegios.

### Setup de la politica administrada

Definir una funcion para formatear todo correctamente

In [ ]:
def jdump(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, sort_keys=True, indent=4 * ' ')

Crear la politica

In [ ]:
# permission-policy.json
iam_permission_policy_for_s3 = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "PermissionPolicyForS3Access",
      "Effect": "Allow",
      "Action":["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource": ["arn:aws:s3:::{}/*".format(mybucket1), 
                   "arn:aws:s3:::{}".format(mybucket1),
                  "arn:aws:s3:::{}/*".format(mybucket2), 
                   "arn:aws:s3:::{}".format(mybucket2)]
    }]
}
jdump(iam_permission_policy_for_s3 , "iam_permission_policy_for_s3.json")

Ya que queremos adjuntar esta politica a multiples roles, debemos crear una politica administrada, en vez de una politica de linea, la cual esta incrustada en nuestro rol.

In [ ]:
!aws --profile $perfilA iam create-policy --policy-name iam_permission_policy_for_s3_$rand \
                                  --path "/aws-labs/" \
                                  --policy-document file://iam_permission_policy_for_s3.json

Debemos crear la politica en ambas cuentas de AWS, por ende repetimos para B

In [ ]:
!aws --profile $perfilB iam create-policy --policy-name iam_permission_policy_for_s3_$rand \
                                --path "/aws-labs/" \
                                --policy-document file://iam_permission_policy_for_s3.json

# Ejercicios

## 1. Confirmar que los roles sin permisos no pueden acceder a baldes sin politicas

Hasta ahora, hemos creado los roles y las politicas. Cuando creamos un rol, debemos incluir la politica de confianza de assume-role, lo que nos dice quien puede asumir dicho rol. Pero por ahora es solo un contenedor vacio, para que nosotros rellenemos con politicas de permisos. Antes de adjuntar una politica de balde, veamos que podemos hacer con los roles asi como estan.

In [ ]:
!awsas --profile $perfilA $rolA1 s3 ls $mybucket1

No mucho, verdad? Veamos que podemos hacer a continuacion

## 2. Chequear que un rol sin permisos puede acceder a un balde con la politica correcta

Creamos la politica

In [ ]:
# mybucket1_policy.json
mybucket1_policy = {
  "Version": "2012-10-17",
  "Statement": [{
      "Sid": "AssumeRolePolicyForS3ReaderRoleB",
      "Effect": "Allow",
      "Principal": {"AWS": ["arn:aws:iam::{}:root".format(accountA), 
                            "arn:aws:iam::{}:role/aws-labs/{}".format(accountA, roleA2)]},
      "Action": ["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource": ["arn:aws:s3:::{}/*".format(mybucket1), 
                   "arn:aws:s3:::{}".format(mybucket1)]

    }]
}
jdump(mybucket1_policy , "mybucket1_policy.json")
mybucket1_policy_str = "'"+json.dumps(mybucket1_policy).replace(' ', '')+"'"

Genial, ahora adjuntemos esta politica de permisos a mybucket1. Notar que aun no tenemos politicas de permisos adjuntas en ningun rol.

In [ ]:
!aws --profile $perfilA s3api put-bucket-policy --bucket $mybucket1 --policy $mybucket1_policy_str

Probar si rolA1 puede acceder

In [ ]:
!awsas --profile $perfilA $rolA1 s3 ls s3://$mybucket1

Esperado:

An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

Aqui puede verse que se nos niega el acceso

Probemos lo mismo con el rolA2, que esta explicitamente permitido en la politica de balde, aunque el rolA2 no tiene una politica de permisos adjunta aun.

In [ ]:
!awsas --profile $perfilA $rolA2 s3 ls s3://$mybucket1

La marca temporal no importa, solo el nombre del archivo que demuestra que nuestro comando funciono.

### Conclusion

Permitir "root" en una politica de S3 no otorga acceso a todos en la cuenta. Sin embargo, permitir explicitamente a un rol en la politica de S3 permite el acceso incluso si el rol no tiene permisos adjuntos. Esto es a lo que nos referimos cuando decimos que "Si el Principal y el Recurso estan en la misma cuenta, entonces el permiso es la union de las politicas adjuntas al Principal y al Recurso".

## 3. Puede un rol con permisos acceder a un balde sin politica de balde?

Ahora, adjuntemos la politica iam_permission_policy_for_s3 al rolA1. Para esto, necesitamos el ARN de la politica. Este se encuentra en nuestras variables fuente, pero podemos obtenerlo de la siguiente manera:

In [ ]:
!aws --profile $perfilA iam list-policies --path-prefix /aws-labs/ | jq -r '.Policies[].Arn'

Y luego pegado a continuacion:

In [ ]:
iam_permission_policy_for_s3_arn = "El ARN va aqui"

Adjuntar la politica al rol

In [ ]:
!aws --profile $pefilA iam attach-role-policy --role-name $rolA1 --policy-arn $iam_permission_policy_for_s3_arn

Ahora, verifiquemos que el rolA1 puede acceder a mybucket1

In [ ]:
!awsas --profile $perfilA $rolA1 s3 ls s3://$mybucket1

Esperar algo como

2020-05-03 13:57:39 185 demo-vars.sh

Nuevamente, la marca temporal no importa, solo el nombre del archivo, que demuestra que el balde es accesible.

Ahora probemos que el rolA1, que tiene una politica de permisos de IAM que le permite trabajar con mybucket1 y mybucket2, no requiere que este ultimo tenga una politica de balde permitiendole hacer eso. Para hacerlo, simplemente intentamos listar los contenidos de mybucket2.

In [ ]:
!awsas --profile $perfilA $rolA1 s3 ls s3://$mybucket2

Esperar algo asi

2020-05-07 13:12:38 227 assume_A_policy.json

### Conclusion

Confirmamos que una politica de IAM adjunta a un rol es todo lo que se requiere para acceder a un balde, afirmando aun mas la regla de "Union dentro de una cuenta".

## 4. Puede el rolB1 con permiso explicito en una politica de IAM acceder a mybucket1 en nuestra cuenta_A?

Ahora, queremos comprobar si podemos acceder a mybucket1 en accountA, introduciendo una politica de rol especifica que nos permita hacer justo eso.

Primero, recordemos la politica actual de mybucket1

In [ ]:
mybucket1_policy

Como dice, solo otorga acceso a accountA y a rolA2 a nuestro balde

Ahora, adjuntemos nuestra politica iam_permission_policy_for_s3 al rolB1. Para eso, necesitamos su ARN, el cual podemos obtener de la siguiente manera:

In [ ]:
!aws --profile $perfilB iam list-policies --path-prefix /aws-labs/ | jq -r '.Policies[].Arn'

Luego, lo pegamos debajo

In [ ]:
iam_permission_policy_for_s3_arnB = "Aqui"

Lo adjuntamos a nuestro rol

In [ ]:
!aws --profile $perfilB iam attach-role-policy --role-name $rolB1 --policy-arn $iam_permission_policy_for_s3_arnB

Finalmente, intentamos acceder al balde

In [ ]:
!awsas --profile $perfilB $rolB1 s3 ls s3://$mybucket1

Esperar

An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

Ya que el rolB1 se encuentra en una cuenta diferente que mybucket1, tanto la politica del recurso como la politica de permiso de IAM deben explicitamente otorgar acceso.

#### Ahora, probemos con mybucket2 en accountA, luego de adjuntar una politica de acceso entre cuentas.

Primero, creamos  la politica

In [ ]:
# mybucket2_policy.json
mybucket2_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:PutObject",
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Effect": "Allow",
            "Principal": {
                "AWS": [
                    "arn:aws:iam::{}:root".format(accountB),
                    "arn:aws:iam::{}:role/aws-labs/{}".format(accountB,roleB2)
                ]
            },
            "Resource": [
                "arn:aws:s3:::{}/*".format(mybucket2),
                "arn:aws:s3:::{}".format(mybucket2)
            ],
            "Sid": "BucketPolicyForS3MyBucket2"
        }
    ]
    
}
mybucket2_policy_str = "'"+json.dumps(mybucket2_policy)+"'"

Luego la ponemos en el balde

In [ ]:
!aws --profile $perfilA s3api put-bucket-policy --bucket $mybucket2 --policy $mybucket2_policy_str

Y luego intentar listar sus contenidos desde el rolB1

In [ ]:
!awsas --profile $perfilB $rolB1 s3 ls s3://$mybucket2

Esperar exito. Esto funciona porque mybucket1 confia en el root de accountB, lo que significa que confia en el administrador de accountB para asignar permisos de S3, INCLUYENDO el acceso a cuentas externas.

Que pasa con el rolB2 entonces? No hemos adjuntado una politica de permisos aun, pero esta explicitamente autorizado en la politica de recurso de mybucket2.

In [ ]:
!awsas --profile $perfilB $rolB2 s3 ls s3://$mybucket2

Esperar AccessDenied. Sorpresa! Este resultado no es consistente con el resultado que se obtiene cuando el rol y el balde de S3 estan en la misma cuenta. Para acceso entre cuentas, incluso cuando el balde de S3 explicitamente autoriza a otra cuenta, el rol tambien debe tener los permisos adjuntos.

### Conclusion

No pudimos acceder a un balde en otra cuenta sin un permiso explicito dentro de la politica de recurso del balde Y de la politica de permisos de IAM. Eso esta perfectamente bien, ya que si nos dejara, podriamos acceder a cualquier balde en cualquier cuenta, solo sabiendo el nombre del mismo.

# 5. Cuidado con las politicas condicionales

Reemplacemos la politica existente de mybucket2 con la politica siguiente. Añade un bloque condicional para solo permitir acceso a ciertas IPs.

In [ ]:
# mybucket2_conditional_policy.json
mybucket2_conditional_policy = {
  "Version":"2012-10-17",
  "Statement":[
    {
      "Sid":"AddCrossAccountPutPolicy",
      "Effect":"Allow",
      "Principal": {"AWS": ["arn:aws:iam::{}:root".format(accountB)]},
      "Action":["s3:PutObject","s3:GetObject","s3:ListBucket"],
      "Resource":["arn:aws:s3:::{}/*".format(mybucket2), "arn:aws:s3:::{}".format(mybucket2)],
      "Condition": {
        "IpAddress": {
          "aws:SourceIp": [
            "54.240.144.0/32",
            "54.240.144.0/24"
          ]
        }
        }
      }
    ]
}
mybucket2_conditional_policy_str = "'"+json.dumps(mybucket2_conditional_policy)+"'"

In [ ]:
!aws --profile $perfilA s3api put-bucket-policy --bucket $mybucket2 --policy $mybucket2_conditional_policy_str

#### Pregunta:
* Podemos listar items del balde desde IPs no permitidas?

Primero, intentemos acceder al balde desde el rolB1, que, como sabemos, podia acceder antes.

In [ ]:
!awsas --profile $perfilB $rolB1 s3 ls s3://$mybucket2

Ahora ya no podemos acceder, porque nuestra IP no esta en la lista de permitidos. 

Que pasa con el rolA1?

In [ ]:
!awsas --profile $perfilA $rolA1 s3 ls s3://$mybucket2

### Conclusion
El rolA1 puede acceder al balde porque su politica de permisos de IAM no esta restringida, y la decision se basa en la union de lo que esta politica permite (si), y lo que la politica del balde permite (no). En cambio, para el rolB1, la politica de IAM (si), intersecta con la politica de recurso (no), llevando a una negacion de acceso.

# 6. La necesidad de declaraciones explicitas de negación

Y si quisieramos solo permitir que rolA1 acceda a mybucket2? Podrias intentar aplicar roles granulares a todos los principals en accountA, para nunca otorgar acceso a un recurso * para operaciones de S3 a ningun principal, pero esto es dificil de hacer cumplir. Una mejor forma es aplicar un Deny explicito a todos los principals excepto rolA1 en la politica de recurso del balde.

Para eso, necesitamos una nueva politica en mybucket2, la cual crearemos a continuación

In [ ]:
mybucket2_deny_policy = {
    "Version": "2012-10-17",
    "Statement": [
      {
        "Sid": "ExplicitDenyBucketPolicyForAllBut",
        "Effect": "Deny",
        "Principal": {
          "AWS": ["arn:aws:iam::{}:role/aws-labs/{}".format(accountB,roleB2),
                  "arn:aws:iam::{}:role/aws-labs/{}".format(accountA,roleA2)]
         },
        "Action": ["s3:PutObject", "s3:GetObject", "s3:ListBucket"],
        "Resource": [ "arn:aws:s3:::{}/*".format(mybucket2),
                      "arn:aws:s3:::{}".format(mybucket2)
        ],
        "Condition": {"ArnNotLike": {
              "aws:SourceArn": ["arn:aws:iam::{}:role/aws-labs/{}".format(accountA,roleA2),
                                "arn:aws:iam::{}:role/aws-labs/{}".format(accountB,roleB2)]}
        }
      },
      {
        "Sid": "AllowCrossAccountForS3MyBucket2",
        "Effect": "Allow",
        "Principal": {
          "AWS": ["arn:aws:iam::{}:role/aws-labs/{}".format(accountB,roleB1)]
         },
        "Action": ["s3:PutObject", "s3:GetObject", "s3:ListBucket"],
        "Resource": [ "arn:aws:s3:::{}/*".format(mybucket2),
                      "arn:aws:s3:::{}".format(mybucket2)
        ]
      }
    ]
  }
mybucket2_deny_policy_str = "'"+json.dumps(mybucket2_deny_policy)+"'"

Luego, la pondremos en el balde, reemplazando la politica anterior

In [ ]:
!aws --profile $perfilA s3api put-bucket-policy --bucket $mybucket2 --policy $mybucket2_deny_policy_str

Ahora, intentemos leer los contenidos del balde desde el rolA1

In [ ]:
!awsas --profile $perfilA $rolA1 s3 ls s3://$mybucket2

Genial! Nuestra politica nos permite hacer eso. Ahora, intentemos desde el rolA2

In [ ]:
!awsas --profile $perfilA $rolA2 s3 ls s3://$mybucket2

Por ahora, todo bien. Solo el rolA1 puede acceder. Ahora, notas la segunda declaracion en la politica? Permite que rolB1 desde accountB pueda acceder a nuestro balde. Si esa declaracion es borrada, entonces solo rolA1 podra acceder. Intentemos acceder al balde desde rolB1 para ver si funciona.

In [ ]:
!awsas --profile $perfilB $rolB1 s3 ls s3://$mybucket2

Solo por si acaso, intentemos acceder desde rolB2

In [ ]:
!awsas --profile $perfilB $rolB2 s3 ls s3://$mybucket2

No nos deja, como planeamos

### Conclusion
Si lo necesitaramos, podemos crear una politica de balde que niega el acceso a todos los principals, excepto el que nosotros queramos, lo cual es bueno para escalabilidad y buenas practicas de seguridad

# Felicitaciones!
Has completado el Lab2

Resumiremos los resultados de la siguiente manera:
* Cuando un rol y un recurso estan en la misma cuenta, el acceso es otorgado si el rol o el recurso lo permiten. Esto se llama unión.
* Cuando un rol y un recurso se encuentran en diferentes cuentas, el permiso de acceso debe ser otorgado tanto por el rol como por el recurso. Esto se llama intersección.

Ejecuta la celda a continuacion para borrar todo desde ambas cuentas, para mantenerlas lo mas limpias posible, siguiendo buenas practicas de seguridad.

Precaucion: el comando s3 rb borrará todos los contenidos de ambos baldes, y luego los eliminará.

In [ ]:
!aws --profile $perfilA iam detach-role-policy --role-name $rolA1 --policy-arn $iam_permission_policy_for_s3_arn
!aws --profile $perfilB iam detach-role-policy --role-name $rolB1 --policy-arn $iam_permission_policy_for_s3_arnB
!aws --profile $perfilA iam delete-policy --policy-arn $iam_permission_policy_for_s3_arn
!aws --profile $perfilB iam delete-policy --policy-arn $iam_permission_policy_for_s3_arnB
!aws --profile $perfilA iam delete-role --role-name $rolA1
!aws --profile $perfilB iam delete-role --role-name $rolB1
!aws --profile $perfilA iam delete-role --role-name $rolA2
!aws --profile $perfilB iam delete-role --role-name $rolB2
!aws --profile $perfilA s3 rb s3://$mybucket1 --force
!aws --profile $perfilA s3 rb s3://$mybucket2 --force